# Semanttinen Ydin

Tässä koodiesimerkissä käytät [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI-kehystä luodaksesi yksinkertaisen agentin.

Tämän esimerkin tavoitteena on näyttää sinulle vaiheet, joita käytämme myöhemmin lisäkoodiesimerkeissä, kun toteutamme erilaisia agenttimallien toimintatapoja.


## Tuo tarvittavat Python-paketit


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Asiakkaan luominen

Tässä esimerkissä käytämme [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) -palvelua saadaksemme pääsyn LLM:ään.

`ai_model_id` on määritelty arvoksi `gpt-4o-mini`. Kokeile vaihtaa malli johonkin toiseen GitHub Models -markkinapaikalla saatavilla olevaan malliin nähdäksesi erilaisia tuloksia.

Jotta voimme käyttää `Azure Inference SDK`:ta, jota käytetään GitHub Models -palvelun `base_url`-osoitteessa, hyödynnämme Semantic Kernelin `OpenAIChatCompletion`-liitintä. Semantic Kernelille on myös muita [saatavilla olevia liittimiä](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion), joita voi käyttää muiden mallipalveluntarjoajien kanssa.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Agentin luominen

Alla luomme agentin nimeltä `TravelAgent`.

Tässä esimerkissä käytämme hyvin yksinkertaisia ohjeita. Voit muuttaa näitä ohjeita nähdäksesi, miten agentti reagoi eri tavoin.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Agentin käynnistäminen

Nyt voimme käynnistää Agentin määrittämällä säikeen tyyppiä `ChatHistoryAgentThread`. Kaikki tarvittavat järjestelmäviestit annetaan agentin invoke_stream-metodin `messages`-avainsanalle.

Kun nämä on määritelty, luomme `user_inputs`, joka edustaa käyttäjän agentille lähettämää viestiä. Tässä tapauksessa olemme asettaneet viestin muotoon `Plan me a sunny vacation`.

Voit vapaasti muuttaa tätä viestiä nähdäksesi, miten agentti reagoi eri tavoin.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
